In [0]:
import keras
import numpy as np
import pandas as pd
import cv2
from keras import backend as K
from keras.layers.merge import concatenate
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten,Input,concatenate
from keras.layers import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D, LocallyConnected1D
from keras.utils import np_utils
#from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam, SGD, RMSprop, Adagrad
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,matthews_corrcoef
import itertools

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/Moshiur26/Datasets/master/Promoter/New%20Dataset/promoter1234.csv",header=None)
data=np.asarray(data)
data = np.delete(data, (0), axis=0)
data = np.delete(data, (0), axis=1)
X=data[:,:data.shape[1]-1]
y=data[:,-1:]
y = np.unique(y, return_inverse=True)[1].tolist()
ys=y
X = to_categorical(X)
y = to_categorical(y)
#X = X.reshape(5720, 81, 1)
X.shape


(5720, 81, 4)

In [0]:
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.15)
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size=0.15)

In [4]:
#LocallyConnected1D
k=4
visible1 = Input(shape=(81,4))
model11=Conv1D(81, kernel_size=k, activation='relu')(visible1)
#drop11=Dropout(0.2)(model11)
pool11=MaxPooling1D(pool_size=3)(model11)#(drop11)
con11=Conv1D(64, kernel_size=k, activation='relu')(pool11)
drop12=Dropout(0.3)(con11)
pool12=MaxPooling1D(pool_size=3)(drop12)
con12=Conv1D(32, kernel_size=k, activation='relu')(pool12)
drop13=Dropout(0.4)(con12)
con13=Conv1D(16, kernel_size=k, activation='relu')(drop13)
drop14=Dropout(0.6)(con13)

#on14=Conv1D(10,kernel_size=k, activation='relu')(drop14)
fltn1=Flatten()(drop14)

k=3
visible2 = Input(shape=(81,4))
model21=Conv1D(81, kernel_size=k, activation='relu')(visible2)
drop21=Dropout(0.2)(model21)
pool21=MaxPooling1D(pool_size=3)(drop21)
con21=Conv1D(64, kernel_size=k, activation='relu')(pool21)
drop21=Dropout(0.2)(con21)
pool22=MaxPooling1D(pool_size=3)(drop21)
con22=Conv1D(32, kernel_size=k, activation='relu')(pool22)
drop22=Dropout(0.4)(con22)
con23=Conv1D(16, kernel_size=k, activation='relu')(drop22)
drop23=Dropout(0.6)(con23)
fltn2=Flatten()(drop23)

k=5
visible3 = Input(shape=(81,4))
model31=Conv1D(81, kernel_size=k, activation='relu')(visible3)
drop31=Dropout(0.2)(model31)
pool31=MaxPooling1D(pool_size=2)(drop31)
con31=Conv1D(64, kernel_size=k, activation='relu')(pool31)
drop32=Dropout(0.4)(con31)
pool32=MaxPooling1D(pool_size=2)(drop32)
con32=Conv1D(32, kernel_size=k, activation='relu')(pool32)
drop33=Dropout(0.6)(con32)
fltn3=Flatten()(drop33)


k=6
visible4 = Input(shape=(81,4))
model41=Conv1D(81, kernel_size=k, activation='relu')(visible4)
drop41=Dropout(0.2)(model41)
pool41=MaxPooling1D(pool_size=2)(drop41)
con41=Conv1D(64, kernel_size=k, activation='relu')(pool41)
drop42=Dropout(0.4)(con41)
pool42=MaxPooling1D(pool_size=2)(drop42)
con42=Conv1D(32, kernel_size=k, activation='relu')(pool42)
drop43=Dropout(0.6)(con42)
fltn4=Flatten()(drop43)



marge=concatenate([fltn1, fltn2, fltn3, fltn4])
#model.add(concatenate([fltn1, fltn2]))
dns0a=Dense(32, activation='relu')(marge)
dns0=Dense(16, activation='relu')(dns0a)
dns1=Dense(8, activation='relu')(dns0)
dns2=Dense(2, activation='softmax')(dns1)

model = Model(input=[visible1, visible2, visible3, visible4], output=dns2)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 81, 4)        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 81, 4)        0                                            
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 79, 81)       1053        input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=22,batch_size=50, verbose=1)
#model.fit([X_train,X_train], y_train,validation_data=([X_val,X_val], y_val),nb_epoch=10, batch_size=28)
model.fit([X_train,X_train,X_train,X_train], y_train,validation_data=([X_val,X_val,X_val,X_val], y_val),epochs=22,batch_size=50, verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 4132 samples, validate on 730 samples
Epoch 1/22
4132/4132 [==============================] - 9s 2ms/step - loss: 0.6151 - acc: 0.6476 - val_loss: 0.5365 - val_acc: 0.7630
Epoch 2/22
4132/4132 [==============================] - 7s 2ms/step - loss: 0.5098 - acc: 0.7631 - val_loss: 0.4966 - val_acc: 0.7589
Epoch 3/22
4132/4132 [==============================] - 7s 2ms/step - loss: 0.4910 - acc: 0.7706 - val_loss: 0.5120 - val_acc: 0.7370
Epoch 4/22
4132/4132 [==============================] - 7s 2ms/step - loss: 0.4783 - acc: 0.7786 - val_loss: 0.4634 - val_acc: 0.7726
Epoch 5/22
4132/4132 [==============================] - 7s 2ms/step - loss: 0.4526 - acc: 0.7911 - val_loss: 0.4559 - val_acc: 0.7781
Epoch 6/22
4132/4132 [==============================] - 7s 2ms/step - loss: 0.4435 - acc: 0.8025 - val_loss: 0.4351 - val_acc: 0.8041
Epoch 7/22
4132/4132 [==============================] - 7s 2ms/step - loss: 0.4114 - acc: 0.8182 - va

In [6]:
y_pred = model.predict([X_test,X_test,X_test,X_test])
print(y_pred.shape)

y_true = np.argmax(y_test, axis=1)
print(y_true.shape)
y_pred = np.argmax(y_pred, axis=1)

(858, 2)
(858,)


In [12]:
print(accuracy_score(y_true, y_pred) * 100)
print(classification_report(y_true, y_pred))


print('Acc: ', accuracy_score(y_true, y_pred) * 100)
print('MCC: ', matthews_corrcoef(y_true, y_pred) *100)

cm = confusion_matrix(y_true, y_pred)
print('Sn:  ', (cm[1,1] / (cm[1,1] + cm[1,0])) * 100)
print('Sp:  ', (cm[0,0] / (cm[0,0] + cm[0,1])) * 100)


87.17948717948718
              precision    recall  f1-score   support

           0       0.83      0.92      0.87       412
           1       0.92      0.82      0.87       446

   micro avg       0.87      0.87      0.87       858
   macro avg       0.88      0.87      0.87       858
weighted avg       0.88      0.87      0.87       858

Acc:  87.17948717948718
MCC:  74.89980727493707
Sn:   82.28699551569507
Sp:   92.47572815533981


In [14]:
scores = model.evaluate([X_test,X_test,X_test,X_test], y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 87.18%


In [9]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#score = loaded_model.evaluate(X, Y, verbose=0)
score = loaded_model.evaluate([X_test,X_test,X_test,X_test], y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

NameError: ignored